In [13]:
!uv add transformers datasets evaluate accelerate scikit-learn kaggle nbformat

Resolved 117 packages in 3.03s
Prepared 46 packages in 42.64s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 47 packages in 5.55s
 + accelerate==1.11.0
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + anyio==4.11.0
 + attrs==25.4.0
 + bleach==6.3.0
 + datasets==4.3.0
 + dill==0.4.0
 + evaluate==0.4.6
 + fastjsonschema==2.21.2
 + filelock==3.20.0
 + frozenlist==1.8.0
 + fsspec==2025.9.0
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + huggingface-hub==0.36.0
 + jinja2==3.1.6
 + jsonschema==4.25.1
 + jsonschema-specifications==2025.9.1
 + kaggle==1.7.4.5
 + markupsafe==3.0.3
 + mpmath==1.3.0
 + multidict==6.7.0
 + multiprocess==0.70.16
 + nbformat==5.10.4
 + networkx==3.5
 + propcache==0.4.1
 + protobuf==6.33.0
 + pyarrow==22.0.0
 + python-slugify==8.0.4
 + pyyaml==6.0.3
 + refe

In [6]:
!uv add scikit-learn

Resolved 53 packages in 1.12s
Prepared 3 packages in 10.62s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 3 packages in 719ms
 + scikit-learn==1.7.2
 + scipy==1.16.3
 + threadpoolctl==3.6.0


In [ ]:

# Download the Kaggle dataset (requires kaggle.json API token)
# If you already have the CSV, skip this download step and set `local_csv_path` accordingly.

import os
from pathlib import Path
local_csv_path = 'data.csv'

print('Using:', local_csv_path)


Using: data.csv


In [10]:

# Quick peek at the dataset
import pandas as pd
print('Reading:', local_csv_path)
df = pd.read_csv(local_csv_path)
print('Shape:', df.shape)
display(df.head(10))
print('\nColumns:', df.columns.tolist())


Reading: data.csv
Shape: (5842, 2)


,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
5,$SPY wouldn't be surprised to see a green close,positive
6,Shell's $70 Billion BG Deal Meets Shareholder ...,negative
7,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative
8,Kone 's net sales rose by some 14 % year-on-ye...,positive
9,The Stockmann department store will have a tot...,neutral



Columns: ['Sentence', 'Sentiment']


In [11]:
id2label = getattr(model.config, "id2label", None)

In [12]:
id2label

{0: 'positive', 1: 'negative', 2: 'neutral'}

In [ ]:

from sklearn.model_selection import train_test_split
import numpy as np

text_cols = ['Sentence']
label_cols = ['Sentiment']

print('Text columns candidates:', text_cols)
print('Label columns candidates:', label_cols)

if len(text_cols) == 0:
    raise ValueError('No text-like column detected. Inspect df.columns and update the notebook to select the correct column.')
if len(label_cols) == 0:
    label_cols = [c for c in df.columns if c.lower() in ('polarity','sent','sentiment','label')]

text_col = text_cols[0]
label_col = label_cols[0]

print('Using text column:', text_col, 'and label column:', label_col)

def normalize_label(x):
    if isinstance(x, str):
        s = x.lower().strip()
        if s in ('negative','neg','-1'): return 1
        if s in ('neutral','neut','0'): return 2
        if s in ('positive','pos','1'): return 0
    try:
        xi = int(x)
        if xi == -1: return 0
        if xi == 0: return 1
        if xi == 1: return 2
    except:
        pass
    return 1

df['label'] = df[label_col].apply(normalize_label)
df = df.rename(columns={text_col: 'text'})
print('After normalization, label distribution:')
print(df['label'].value_counts())

trainval, test = train_test_split(df, test_size=0.10, stratify=df['label'], random_state=42)
train, val = train_test_split(trainval, test_size=0.1111, stratify=trainval['label'], random_state=42)  # ~0.10 of total
print('Sizes -> train:', len(train), 'val:', len(val), 'test:', len(test))


Text columns candidates: ['Sentence']
Label columns candidates: ['Sentiment']
Using text column: Sentence and label column: Sentiment
After normalization, label distribution:
label
2    3130
0    1852
1     860
Name: count, dtype: int64
Sizes -> train: 4672 val: 585 test: 585


In [ ]:
!uv add transformers datasets 

In [ ]:

from datasets import Dataset, DatasetDict
train_ds = Dataset.from_pandas(train.reset_index(drop=True))
val_ds = Dataset.from_pandas(val.reset_index(drop=True))
test_ds = Dataset.from_pandas(test.reset_index(drop=True))
dataset = DatasetDict({'train': train_ds, 'validation': val_ds, 'test': test_ds})

print(dataset)
print(dataset['train'][0])

model_name = 'ProsusAI/finbert'
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print('Tokenizer loaded for', model_name)

def preprocess_fn(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized = dataset.map(preprocess_fn, batched=True)
tokenized = tokenized.remove_columns(['text', 'index', 'label'] if 'index' in tokenized['train'].column_names else ['text'])
tokenized = tokenized.rename_column('label', 'labels') if 'labels' not in tokenized['train'].column_names else tokenized
tokenized.set_format('torch')


DatasetDict({
    train: Dataset({
        features: ['text', 'Sentiment', 'label'],
        num_rows: 4672
    })
    validation: Dataset({
        features: ['text', 'Sentiment', 'label'],
        num_rows: 585
    })
    test: Dataset({
        features: ['text', 'Sentiment', 'label'],
        num_rows: 585
    })
})
{'text': 'Mr. Koistinen joins from Nokia Siemens Networks where he has held various senior sales management and business development positions since 1997 .', 'Sentiment': 'neutral', 'label': 2}
Tokenizer loaded for ProsusAI/finbert


Map: 100%|██████████| 585/585 [00:00<00:00, 17093.73 examples/s]


In [ ]:

from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix

print('Loading pretrained pipeline (this downloads the HF model & tokenizer).')
pipe = pipeline('text-classification', model=model_name, tokenizer=model_name, return_all_scores=True, device=0)

label_map = {'negative':1, 'neutral':2, 'positive':0}

def hf_predict(texts, batch_size=64):
    preds = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        outs = pipe(batch)
        for o in outs:
            best = max(o, key=lambda x: x['score'])
            lbl = best['label'].lower()
            preds.append(label_map.get(lbl, 1))
    return preds

test_texts = test['text'].tolist()
y_true = test['label'].tolist()
print('Running predictions (pretrained finbert) on', len(test_texts), 'examples...')
y_pred_pre = hf_predict(test_texts)

print('Pretrained model metrics:')
acc = accuracy_score(y_true, y_pred_pre)
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred_pre, average='weighted', zero_division=0)
print('Accuracy:', acc)
print('Precision:', prec, 'Recall:', rec, 'F1:', f1)
print('\nClassification report:\n', classification_report(y_true, y_pred_pre, target_names=['neg','neu','pos']))
print('\nConfusion matrix:\n', confusion_matrix(y_true, y_pred_pre))


Loading pretrained pipeline (this downloads the HF model & tokenizer).


Device set to use cuda:0


Running predictions (pretrained finbert) on 585 examples...
Pretrained model metrics:
Accuracy: 0.7367521367521368
Precision: 0.7718479166161285 Recall: 0.7367521367521368 F1: 0.7459446922158657

Classification report:
               precision    recall  f1-score   support

         neg       0.89      0.72      0.80       186
         neu       0.47      0.77      0.58        86
         pos       0.79      0.74      0.76       313

    accuracy                           0.74       585
   macro avg       0.71      0.74      0.71       585
weighted avg       0.77      0.74      0.75       585


Confusion matrix:
 [[134   8  44]
 [  1  66  19]
 [ 16  66 231]]


In [8]:
!pip install evaluate

  Using cached evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
Using cached evaluate-0.4.6-py3-none-any.whl (84 kB)



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
import evaluate
from sklearn.metrics import precision_recall_fscore_support
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
import evaluate
from sklearn.metrics import precision_recall_fscore_support

print("torch.cuda.is_available():", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())

device_id = 0 if torch.cuda.is_available() else -1  

num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
if torch.cuda.is_available():
    model.to(torch.device(f"cuda:{device_id}"))

metric_acc = evaluate.load('accuracy')
metric_f1 = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = metric_acc.compute(predictions=preds, references=labels)['accuracy']
    f1 = metric_f1.compute(predictions=preds, references=labels, average='weighted')['f1']
    prec, rec, f1s, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    return {'accuracy': acc, 'f1': f1, 'precision': prec, 'recall': rec}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='finetuned-finbert-output',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,   
    per_device_eval_batch_size=32,   
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    fp16=True if torch.cuda.is_available() else False,  
    logging_steps=50,
    dataloader_num_workers=4,         
    gradient_accumulation_steps=1   
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print('Starting training... (will use GPU if available)')
trainer.train()
print('Training completed.')
# Move model back to CPU for safe saving if needed
model.to('cpu')


torch.cuda.is_available(): True
GPU count: 1
Current device: 0


C:\Users\avane\AppData\Local\Temp\ipykernel_19072\218703820.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training... (will use GPU if available)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.384200,0.487517,0.782906,0.785281,0.813833,0.782906
2,0.286500,0.410227,0.815385,0.820998,0.838706,0.815385
3,0.178800,0.465082,0.801709,0.796808,0.793776,0.801709
4,0.163400,0.527488,0.805128,0.793392,0.788447,0.805128
5,0.148500,0.519556,0.798291,0.792597,0.788533,0.798291


Training completed.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenized.set_format(type='torch', device=device)

trainer.model.to(device)

print('Evaluating fine-tuned model on tokenized test set...')
metrics = trainer.evaluate(eval_dataset=tokenized['test'])
print('Eval metrics (HF Trainer):', metrics)

preds_output = trainer.predict(tokenized['test'])
preds = np.argmax(preds_output.predictions, axis=-1)
y_true = test['label'].tolist()

print('\nFine-tuned model classification report:')
print(classification_report(y_true, preds, target_names=['neg','neu','pos']))

from sklearn.metrics import confusion_matrix
print('\nConfusion matrix:\n', confusion_matrix(y_true, preds))


Evaluating fine-tuned model on tokenized test set...


Eval metrics (HF Trainer): {'eval_loss': 0.359531432390213, 'eval_accuracy': 0.8136752136752137, 'eval_f1': 0.823713687061814, 'eval_precision': 0.8503595485308112, 'eval_recall': 0.8136752136752137, 'eval_runtime': 33.852, 'eval_samples_per_second': 17.281, 'eval_steps_per_second': 0.561, 'epoch': 5.0}

Fine-tuned model classification report:
              precision    recall  f1-score   support

         neg       0.91      0.89      0.90       186
         neu       0.50      0.80      0.62        86
         pos       0.91      0.77      0.84       313

    accuracy                           0.81       585
   macro avg       0.77      0.82      0.78       585
weighted avg       0.85      0.81      0.82       585


Confusion matrix:
 [[165  10  11]
 [  5  69  12]
 [ 12  59 242]]


In [ ]:
# Minimal: authenticate then push saved finetuned directory
import os
from huggingface_hub import login

repo_id = "Avaneeshkarthik/finbert-finetuned"
trainer.save_model("finetuned-finbert")
tokenizer.save_pretrained("finetuned-finbert")




('finetuned-finbert\\tokenizer_config.json',
 'finetuned-finbert\\special_tokens_map.json',
 'finetuned-finbert\\vocab.txt',
 'finetuned-finbert\\added_tokens.json',
 'finetuned-finbert\\tokenizer.json')